In [ ]:
#ENHANCED 
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM

# Assuming your data is in a CSV file
df = pd.read_csv('AAPLenhanced.csv')

# Filling missing values
df.fillna(method='pad', inplace=True)
df.fillna(0, inplace=True)  # Fill remaining NaNs with 0

# Selecting relevant columns
data = df[['Adj.Close', 'Volume', 'SMA_50', 'GSPC.Adjusted', 'rsi']]

# Convert the dataframe to a numpy array
dataset = data.values

# Get the number of rows to train the model on (80% of the data)
training_data_len = int(np.ceil(len(dataset) * 0.8))

# Scale the data to be values between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Scale the 'Adj.Close' data to be values between 0 and 1
close_scaler = MinMaxScaler(feature_range=(0, 1))
close_scaler.fit(df[['Adj.Close']].values)

# Create the scaled training data set
train_data = scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train data sets
sequence_length = 60
x_train = []
y_train = []

for i in range(sequence_length, len(train_data)):
    x_train.append(train_data[i-sequence_length:i, :])
    y_train.append(train_data[i, 0])  # Use only the 'Adj.Close' as the target variable

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)


# Build the BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Prepare the testing data
test_data = scaled_data[training_data_len - sequence_length:, :]
x_test = []
y_test = df['Adj.Close'][training_data_len:, ].values

for i in range(sequence_length, len(test_data)):
    x_test.append(test_data[i-sequence_length:i, :])

x_test = np.array(x_test)

# Reshape the data into the shape accepted by the RNN
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

# Predict the values for the test data
predictions = model.predict(x_test)

# Perform inverse transformation on the predicted values
predicted_adj_close = close_scaler.inverse_transform(predictions).flatten()

# Calculate the evaluation metrics
mse = mean_squared_error(y_test, predicted_adj_close)
mae = mean_absolute_error(y_test, predicted_adj_close)

# Print the evaluation metrics
print('Mean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)

In [ ]:
#RAW OHLCV

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM

# Assuming your data is in a CSV file
df = pd.read_csv('AAPLraw.csv')

# Filling missing values
df.fillna(method='pad', inplace=True)
df.fillna(0, inplace=True)  # Fill remaining NaNs with 0

# Selecting relevant columns
data = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert the dataframe to a numpy array
dataset = data.values

# Get the number of rows to train the model on (80% of the data)
training_data_len = int(np.ceil(len(dataset) * 0.8))

# Scale the data to be values between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Scale the 'Adj.Close' data to be values between 0 and 1
close_scaler = MinMaxScaler(feature_range=(0, 1))
close_scaler.fit(df[['Adj Close']].values)

# Create the scaled training data set
train_data = scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train data sets
sequence_length = 60
x_train = []
y_train = []

for i in range(sequence_length, len(train_data)):
    x_train.append(train_data[i-sequence_length:i, :])
    y_train.append(train_data[i, 0])  # Use only the 'Adj.Close' as the target variable

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)


# Build the BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


# Prepare the testing data
test_data = scaled_data[training_data_len - sequence_length:, :]
x_test = []
y_test = df['Adj Close'][training_data_len:, ].values

for i in range(sequence_length, len(test_data)):
    x_test.append(test_data[i-sequence_length:i, :])

x_test = np.array(x_test)

# Reshape the data into the shape accepted by the RNN
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

# Predict the values for the test data
predictions = model.predict(x_test)

# Perform inverse transformation on the predicted values
predicted_adj_close = close_scaler.inverse_transform(predictions).flatten()

# Calculate the evaluation metrics
mse = mean_squared_error(y_test, predicted_adj_close)
mae = mean_absolute_error(y_test, predicted_adj_close)

# Print the evaluation metrics
print('Mean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)


import matplotlib.pyplot as plt
df['Date'] = pd.to_datetime(df['Date'])

# Plot the actual vs predicted values
plt.figure(figsize=(14,5))
plt.plot(df['Date'][training_data_len:, ], y_test, color = 'red', label = 'Real AAPL Stock Price')
plt.plot(df['Date'][training_data_len:, ], predicted_adj_close, color = 'blue', label = 'Predicted AAPL Stock Price')
plt.title('AAPL Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('AAPL Stock Price')
plt.legend()
plt.show()